In [1]:
import random
from datetime import datetime, timedelta
import pandas as pd

In [2]:
#Веса параметров
weight_parameters = {
    "Местоположение": 0.31,
    "Инфраструктура района": 0.22,
    "Уровень ремонта": 0.17,
    "Тип дома": 0.12,
    "Инфраструктура дома и придомовой территории": 0.07,
    "Этаж": 0.04,
    "Площадь": 0.02,
    "Иные факторы": 0.05
}

In [3]:
#Веса атрибутов
weights_of_parameters = {
    "Местоположение": {
        "Район": 0.33,
        "Метро": 0.44,
        "Транспортная доступность": 0.23,
    },
    
    "Инфраструктура района": {
        "Экология": 0.23,
        "Школы": 0.14,
        "Детские сады": 0.09,
        "Больницы": 0.10,
        "Торговые центры": 0.14,
        "Кафе и рестораны": 0.10,
        "Спорт и отдых": 0.10,
        "Банки": 0.05,
        "Аптеки": 0.05,
    },

    "Уровень квартиры": {
        "Количество комнат": 0.31,
        "Этаж": 0.16,
        "Высота потолков": 0.10,
        "Планировка": 0.16,
        "Состояние": 0.16,
        "Балкон/лоджия": 0.06,
        "Вид из окна": 0.05,
    },

    "Состояние дома": {
        "Тип дома": 0.26,
        "Год постройки": 0.16,
        "Состояние дома": 0.16,
        "Лифт": 0.11,
        "Мусоропровод": 0.05,
        "Парковка": 0.11,
        "Консьерж": 0.05,
        "Домофон/видеодомофон": 0.05,
        "Система безопасности": 0.05,
    },

    "Иные факторы": {
        "Спрос и предложение": 0.45,
        "Экономическая ситуация": 0.27,
        "Политические факторы": 0.18,
        "Сезонность": 0.10
    }
}


In [4]:
# Веса станций метро
weights_stations = {
    "Красная": {
        "Бульвар Рокоссовского": 0.10,
        "Черкизовская": 0.15,
        "Преображенская площадь": 0.15,
        "Сокольники": 0.20,
        "Красносельская": 0.20,
        "Комсомольская": 0.25,
        "Красные Ворота": 0.25,
        "Чистые пруды": 0.25,
        "Лубянка": 0.25,
        "Охотный Ряд": 0.25,
        "Библиотека имени Ленина": 0.25,
        "Кропоткинская": 0.25,
        "Парк культуры": 0.25,
        "Фрунзенская": 0.20,
        "Спортивная": 0.20,
        "Воробьёвы горы": 0.20,
        "Университет": 0.20,
        "Проспект Вернадского": 0.15,
        "Юго-Западная": 0.15,
        "Тропарёво": 0.10,
        "Румянцево": 0.05,
        "Саларьево": 0.05,
        "Филатов Луг": 0.05,
        "Прокшино": 0.05,
        "Ольховая": 0.05,
        "Новомосковская": 0.05,
        "Потапово": 0.05
    },
    "Зеленая": {
        "Ховрино": 0.05,
        "Беломорская": 0.10,
        "Речной вокзал": 0.15,
        "Водный стадион": 0.15,
        "Войковская": 0.20,
        "Сокол": 0.20,
        "Аэропорт": 0.20,
        "Динамо": 0.25,
        "Белорусская": 0.25,
        "Маяковская": 0.25,
        "Тверская": 0.25,
        "Театральная": 0.25,
        "Новокузнецкая": 0.20,
        "Павелецкая": 0.20,
        "Автозаводская": 0.15,
        "Технопарк": 0.15,
        "Коломенская": 0.15,
        "Каширская": 0.10,
        "Кантемировская": 0.10,
        "Царицыно": 0.10,
        "Орехово": 0.10,
        "Домодедовская": 0.10,
        "Красногвардейская": 0.10,
        "Алма-Атинская": 0.05
    },
    "Синяя": {
        "Пятницкое шоссе": 0.05,
        "Митино": 0.10,
        "Волоколамская": 0.10,
        "Мякинино": 0.10,
        "Строгино": 0.15,
        "Крылатское": 0.15,
        "Молодёжная": 0.15,
        "Кунцевская": 0.20,
        "Славянский бульвар": 0.20,
        "Парк Победы": 0.20,
        "Киевская": 0.25,
        "Смоленская": 0.25,
        "Арбатская": 0.25,
        "Площадь Революции": 0.25,
        "Курская": 0.20,
        "Бауманская": 0.20,
        "Электрозаводская": 0.15,
        "Семёновская": 0.15,
        "Партизанская": 0.10,
        "Измайловская": 0.10,
        "Первомайская": 0.10,
        "Щёлковская": 0.05
    },
    "Голубая": {
        "Кунцевская": 0.20,
        "Пионерская": 0.15,
        "Филёвский парк": 0.15,
        "Багратионовская": 0.15,
        "Фили": 0.10,
        "Кутузовская": 0.20,
        "Студенческая": 0.20,
        "Москва-Сити": 0.25,
        "Деловой центр": 0.25,
        "Киевская": 0.25,
        "Смоленская": 0.25,
        "Арбатская": 0.25,
        "Александровский сад": 0.20
    },
    "Коричневая": {
        "Парк культуры": 0.25,
        "Октябрьская": 0.20,
        "Добрынинская": 0.20,
        "Павелецкая": 0.20,
        "Таганская": 0.20,
        "Курская": 0.20,
        "Комсомольская": 0.25,
        "Проспект Мира": 0.15,
        "Новослободская": 0.15,
        "Белорусская": 0.25,
        "Краснопресненская": 0.25,
        "Киевская": 0.25
    },
    "Оранжевая": {
        "Медведково": 0.05,
        "Бабушкинская": 0.10,
        "Свиблово": 0.10,
        "Ботанический сад": 0.15,
        "ВДНХ": 0.20,
        "Алексеевская": 0.15,
        "Рижская": 0.15,
        "Проспект Мира": 0.15,
        "Сухаревская": 0.15,
        "Тургеневская": 0.20,
        "Китай-город": 0.25,
        "Третьяковская": 0.20,
        "Октябрьская": 0.20,
        "Шаболовская": 0.15,
        "Ленинский проспект": 0.15,
        "Академическая": 0.15,
        "Профсоюзная": 0.10,
        "Новые Черёмушки": 0.10,
        "Калужская": 0.10,
        "Беляево": 0.10,
        "Коньково": 0.05,
        "Тёплый Стан": 0.05,
        "Ясенево": 0.05,
        "Новоясеневская": 0.05
    },
    "Фиолетовая": {
        "Планерная": 0.05,
        "Сходненская": 0.10,
        "Тушинская": 0.10,
        "Спартак": 0.10,
        "Щукинская": 0.10,
        "Октябрьское Поле": 0.15,
        "Полежаевская": 0.15,
        "Беговая": 0.15,
        "Улица 1905 года": 0.15,
        "Баррикадная": 0.15,
        "Пушкинская": 0.20,
        "Кузнецкий Мост": 0.20,
        "Китай-город": 0.25,
        "Таганская": 0.20,
        "Пролетарская": 0.10,
        "Волгоградский проспект": 0.10,
        "Текстильщики": 0.10,
        "Кузьминки": 0.10,
        "Рязанский проспект": 0.05,
        "Выхино": 0.05,
        "Лермонтовский проспект": 0.05,
        "Жулебино": 0.05,
        "Котельники": 0.05
    },
    "Желтая": {
        "Третьяковская": 0.20,
        "Марксистская": 0.15,
        "Площадь Ильича": 0.15,
        "Авиамоторная": 0.10,
        "Шоссе Энтузиастов": 0.10,
        "Перово": 0.10,
        "Новогиреево": 0.10,
        "Новокосино": 0.10,
        "Деловой центр": 0.25,
        "Парк Победы": 0.20,
        "Минская": 0.10,
        "Ломоносовский проспект": 0.10,
        "Раменки": 0.10,
        "Мичуринский проспект": 0.10,
        "Озёрная": 0.05,
        "Говорово": 0.05,
        "Солнцево": 0.05,
        "Боровское шоссе": 0.05,
        "Новопеределкино": 0.05,
        "Рассказовка": 0.05,
        "Пыхтино": 0.05,
        "Аэропорт Внуково": 0.05
    },
    "Серый": {
        "Алтуфьево": 0.05,
        "Бибирево": 0.05,
        "Отрадное": 0.05,
        "Владыкино": 0.05,
        "Петровско-Разумовская": 0.05,
        "Тимирязевская": 0.05,
        "Дмитровская": 0.05,
        "Савёловская": 0.10,
        "Менделеевская": 0.10,
        "Цветной бульвар": 0.15,
        "Чеховская": 0.15,
        "Боровицкая": 0.15,
        "Полянка": 0.15,
        "Серпуховская": 0.10,
        "Тульская": 0.10,
        "Нагатинская": 0.10,
        "Нагорная": 0.10,
        "Нахимовский проспект": 0.10,
        "Севастопольская": 0.10,
        "Чертановская": 0.05,
        "Южная": 0.05,
        "Пражская": 0.05,
        "Улица Академика Янгеля": 0.05,
        "Аннино": 0.05,
        "Бульвар Дмитрия Донского": 0.05
    },
    "Салатовая": {
        "Физтех": 0.05,
        "Лианозово": 0.05,
        "Яхромская": 0.05,
        "Селигерская": 0.05,
        "Верхние Лихоборы": 0.05,
        "Окружная": 0.05,
        "Петровско-Разумовская": 0.05,
        "Фонвизинская": 0.05,
        "Бутырская": 0.05,
        "Марьина Роща": 0.05,
        "Достоевская": 0.05,
        "Трубная": 0.05,
        "Сретенский бульвар": 0.05,
        "Чкаловская": 0.05,
        "Римская": 0.05,
        "Крестьянская застава": 0.05,
        "Дубровка": 0.05,
        "Кожуховская": 0.05,
        "Печатники": 0.05,
        "Волжская": 0.05,
        "Люблино": 0.05,
        "Братиславская": 0.05,
        "Марьино": 0.05,
        "Борисово": 0.05,
        "Шипиловская": 0.05,
        "Зябликово": 0.05
    },
    "Бирюзовая": {
        "Савёловская": 0.10,
        "Петровский парк": 0.10,
        "ЦСКА": 0.10,
        "Хорошёвская": 0.10,
        "Народное Ополчение": 0.10,
        "Мнёвники": 0.10,
        "Терехово": 0.10,
        "Кунцевская": 0.20,
        "Давыдково": 0.10,
        "Аминьевская": 0.10,
        "Мичуринский проспект": 0.10,
        "Проспект Вернадского": 0.20,
        "Новаторская": 0.10,
        "Воронцовская": 0.10,
        "Зюзино": 0.05,
        "Каховская": 0.05,
        "Варшавская": 0.05,
        "Каширская": 0.05,
        "Кленовый бульвар": 0.05,
        "Нагатинский Затон": 0.05,
        "Печатники": 0.05,
        "Текстильщики": 0.05,
        "Нижегородская": 0.05,
        "Авиамоторная": 0.05,
        "Лефортово": 0.05,
        "Электрозаводская": 0.05,
        "Сокольники": 0.05,
        "Рижская": 0.05,
        "Марьина Роща": 0.05
    },
    "Лавандовая": {
        "Битцевский парк": 0.05,
        "Лесопарковая": 0.05,
        "Улица Старокачаловская": 0.05,
        "Улица Скобелевская": 0.05,
        "Бульвар Адмирала Ушакова": 0.05,
        "Улица Горчакова": 0.05,
        "Бунинская аллея": 0.05
    },
    "Розовая": {
        "Нижегородская": 0.05,
        "Стахановская": 0.05,
        "Окская": 0.05,
        "Юго-Восточная": 0.05,
        "Косино": 0.05,
        "Улица Дмитриевского": 0.05,
        "Лухмановская": 0.05,
        "Некрасовка": 0.05
    },
    "Фуксия": {
        "Новаторская": 0.05,
        "Университет дружбы народов": 0.05,
        "Генерала Тюленева": 0.05,
        "Тютчевская": 0.05
    }
}

# Функция для комбинирования весов
def combine_weights(weights_parameters, weights_stations, line, station):
    combined_weights = {}
    station_weight = weights_stations[line][station]
    for param, weight in weights_parameters.items():
        combined_weights[param] = weight * (1 + station_weight)
    return combined_weights

In [5]:
# Словарь со ставками рефинансирования ЦБ РФ за каждый месяц с 2018 по 2024 год
cb_rates = {
    2018: {
        1: 7.75,  # Январь
        2: 7.50,  # Февраль
        3: 7.50,  # Март
        4: 7.25,  # Апрель
        5: 7.25,  # Май
        6: 7.25,  # Июнь
        7: 7.25,  # Июль
        8: 7.25,  # Август
        9: 7.50,  # Сентябрь
        10: 7.50, # Октябрь
        11: 7.50, # Ноябрь
        12: 7.75  # Декабрь
    },
    2019: {
        1: 7.75,  # Январь
        2: 7.75,  # Февраль
        3: 7.75,  # Март
        4: 7.75,  # Апрель
        5: 7.75,  # Май
        6: 7.50,  # Июнь
        7: 7.25,  # Июль
        8: 7.25,  # Август
        9: 7.00,  # Сентябрь
        10: 6.50, # Октябрь
        11: 6.50, # Ноябрь
        12: 6.25  # Декабрь
    },
    2020: {
        1: 6.25,  # Январь
        2: 6.00,  # Февраль
        3: 6.00,  # Март
        4: 5.50,  # Апрель
        5: 5.50,  # Май
        6: 5.50,  # Июнь
        7: 4.50,  # Июль
        8: 4.25,  # Август
        9: 4.25,  # Сентябрь
        10: 4.00, # Октябрь
        11: 4.00, # Ноябрь
        12: 4.25  # Декабрь
    },
    2021: {
        1: 4.25,  # Январь
        2: 4.25,  # Февраль
        3: 4.50,  # Март
        4: 5.00,  # Апрель
        5: 5.00,  # Май
        6: 5.50,  # Июнь
        7: 6.50,  # Июль
        8: 6.75,  # Август
        9: 6.75,  # Сентябрь
        10: 7.50, # Октябрь
        11: 7.50, # Ноябрь
        12: 8.50  # Декабрь
    },
    2022: {
        1: 8.50,  # Январь
        2: 9.50,  # Февраль
        3: 20.00, # Март
        4: 17.00, # Апрель
        5: 14.00, # Май
        6: 9.50,  # Июнь
        7: 8.50,  # Июль
        8: 8.00,  # Август
        9: 7.50,  # Сентябрь
        10: 7.50, # Октябрь
        11: 7.50, # Ноябрь
        12: 7.50  # Декабрь
    },
    2023: {
        1: 7.50,  # Январь
        2: 7.50,  # Февраль
        3: 7.50,  # Март
        4: 7.50,  # Апрель
        5: 7.50,  # Май
        6: 7.50,  # Июнь
        7: 7.50,  # Июль
        8: 12.00,  # Август
        9: 12.00,  # Сентябрь
        10: 13.00, # Октябрь
        11: 15.00, # Ноябрь
        12: 16.00  # Декабрь
    },
    2024: {
        1: 16.00,  # Январь
        2: 16.00,  # Февраль
        3: 16.00,  # Март
        4: 16.00,  # Апрель
        5: 16.00,  # Май
        6: 16.00,  # Июнь
        7: 16.00,  # Июль
        8: 18.00,  # Август
        9: 19.00,   # Сентябрь
        10: 21.00  # Октябрь
    }
}

In [6]:
# # Функция для расчета цены квартиры на основе признаков и их весов
# def calculate_price(area, rooms, floor, ceiling_height, layout, condition, balcony, view, house_type, year_built, house_condition, elevator, garbage_chute, parking, concierge, intercom, security_system, schools, kindergartens, hospitals, shopping_centers, cafes_restaurants, sports_recreation, banks, pharmacies, demand_supply, economic_situation, political_factors, seasonality, distance_to_metro, combined_weights):
#     base_price_per_sqm = 85000  # Базовая цена за квадратный метр
#     price = base_price_per_sqm * area
    
#     price *= (1 + combined_weights["Район"])
#     price *= (1 + combined_weights["Метро"])
#     price *= (1 + combined_weights["Транспортная доступность"])
#     price *= (1 + combined_weights["Экология"])
#     price *= (1 + combined_weights["Площадь"])
#     price *= (1 + combined_weights["Количество комнат"])
#     price *= (1 + combined_weights["Этаж"])
#     price *= (1 + combined_weights["Высота потолков"])
#     price *= (1 + combined_weights["Планировка"])
#     price *= (1 + combined_weights["Состояние"])
#     price *= (1 + combined_weights["Балкон/лоджия"])
#     price *= (1 + combined_weights["Вид из окна"])
#     price *= (1 + combined_weights["Тип дома"])
#     price *= (1 + combined_weights["Год постройки"])
#     price *= (1 + combined_weights["Состояние дома"])
#     price *= (1 + combined_weights["Лифт"])
#     price *= (1 + combined_weights["Мусоропровод"])
#     price *= (1 + combined_weights["Парковка"])
#     price *= (1 + combined_weights["Консьерж"])
#     price *= (1 + combined_weights["Домофон/видеодомофон"])
#     price *= (1 + combined_weights["Система безопасности"])
#     price *= (1 + combined_weights["Школы"])
#     price *= (1 + combined_weights["Детские сады"])
#     price *= (1 + combined_weights["Больницы"])
#     price *= (1 + combined_weights["Торговые центры"])
#     price *= (1 + combined_weights["Кафе и рестораны"])
#     price *= (1 + combined_weights["Спорт и отдых"])
#     price *= (1 + combined_weights["Банки"])
#     price *= (1 + combined_weights["Аптеки"])
#     price *= (1 + combined_weights["Спрос и предложение"])
#     price *= (1 + combined_weights["Экономическая ситуация"])
#     price *= (1 + combined_weights["Политические факторы"])
#     price *= (1 + combined_weights["Сезонность"])
    
#     # Учет расстояния до метро
#     price *= (1 - distance_to_metro / 10000)  # Предположим, что влияние расстояния до метро линейно уменьшается с увеличением расстояния
    
#     return price

# # Функция для создания записей о проданных квартирах
# def create_sale_records(num_records, output_file):
#     records = []
#     start_date = datetime(2018, 1, 1)
#     end_date = datetime.now()
#     lines = list(weights_stations.keys())
    
#     for _ in range(num_records):
#         # Случайная дата продажи
#         sale_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
#         year = sale_date.year
#         month = sale_date.month
#         cb_rate = cb_rates[year][month]
        
#         # Случайный район и станция метро
#         line = random.choice(lines)
#         station = random.choice(list(weights_stations[line].keys()))
#         combined_weights = combine_weights(weights_parameters, weights_stations, line, station)
        
#         # Случайные параметры квартиры
#         area = random.randint(30, 150)
#         rooms = random.randint(1, 5)
#         floor = random.randint(1, 20)
#         ceiling_height = random.choice([2.5, 2.7, 3.0])
#         layout = random.choice(["Свободная", "Изолированные комнаты", "Смежные комнаты"])
#         condition = random.choice(["Новостройка", "Евроремонт", "Требует ремонта"])
#         balcony = random.choice(["Да", "Нет"])
#         view = random.choice(["На улицу", "Во двор", "На панораму города"])
#         house_type = random.choice(["Панельный", "Кирпичный", "Монолитный"])
#         year_built = random.randint(1960, 2023)
#         house_condition = random.choice(["Хорошее", "Среднее", "Требует ремонта"])
#         elevator = random.choice(["Да", "Нет"])
#         garbage_chute = random.choice(["Да", "Нет"])
#         parking = random.choice(["Открытая", "Закрытая", "Подземная"])
#         concierge = random.choice(["Да", "Нет"])
#         intercom = random.choice(["Да", "Нет"])
#         security_system = random.choice(["Да", "Нет"])
#         schools = random.choice(["Близко", "Средне", "Далеко"])
#         kindergartens = random.choice(["Близко", "Средне", "Далеко"])
#         hospitals = random.choice(["Близко", "Средне", "Далеко"])
#         shopping_centers = random.choice(["Близко", "Средне", "Далеко"])
#         cafes_restaurants = random.choice(["Близко", "Средне", "Далеко"])
#         sports_recreation = random.choice(["Близко", "Средне", "Далеко"])
#         banks = random.choice(["Близко", "Средне", "Далеко"])
#         pharmacies = random.choice(["Близко", "Средне", "Далеко"])
#         demand_supply = random.choice(["Высокий спрос", "Средний спрос", "Низкий спрос"])
#         economic_situation = random.choice(["Положительная", "Нейтральная", "Негативная"])
#         political_factors = random.choice(["Стабильные", "Нестабильные"])
#         seasonality = random.choice(["Высокий сезон", "Низкий сезон"])
#         distance_to_metro = random.randint(100, 2000)  # Расстояние до метро в метрах
        
#         # Расчет цены квартиры
#         price = calculate_price(area, rooms, floor, ceiling_height, layout, condition, balcony, view, house_type, year_built, house_condition, elevator, garbage_chute, parking, concierge, intercom, security_system, schools, kindergartens, hospitals, shopping_centers, cafes_restaurants, sports_recreation, banks, pharmacies, demand_supply, economic_situation, political_factors, seasonality, distance_to_metro, combined_weights)
        
#         # Создание записи
#         record = {
#             "Дата продажи": sale_date,
#             "Ставка ЦБ": cb_rate,
#             "Район": line,
#             "Станция метро": station,
#             "Площадь": area,
#             "Количество комнат": rooms,
#             "Этаж": floor,
#             "Высота потолков": ceiling_height,
#             "Планировка": layout,
#             "Состояние": condition,
#             "Балкон/лоджия": balcony,
#             "Вид из окна": view,
#             "Тип дома": house_type,
#             "Год постройки": year_built,
#             "Состояние дома": house_condition,
#             "Лифт": elevator,
#             "Мусоропровод": garbage_chute,
#             "Парковка": parking,
#             "Консьерж": concierge,
#             "Домофон/видеодомофон": intercom,
#             "Система безопасности": security_system,
#             "Школы": schools,
#             "Детские сады": kindergartens,
#             "Больницы": hospitals,
#             "Торговые центры": shopping_centers,
#             "Кафе и рестораны": cafes_restaurants,
#             "Спорт и отдых": sports_recreation,
#             "Банки": banks,
#             "Аптеки": pharmacies,
#             "Спрос и предложение": demand_supply,
#             "Экономическая ситуация": economic_situation,
#             "Политические факторы": political_factors,
#             "Сезонность": seasonality,
#             "Расстояние до метро": distance_to_metro,
#             "Цена": price
#         }
        
#         records.append(record)
    
#     # Создание DataFrame и запись в CSV файл
#     df = pd.DataFrame(records)
#     df.to_csv(output_file, index=False)

# # Пример использования
# num_records = 1000000
# output_file = "sale_records.csv"
# create_sale_records(num_records, output_file)

# print(f"Записи успешно записаны в файл {output_file}")

In [7]:
# Веса параметров
weight_parameters = {
    "Местоположение": 0.30,
    "Инфраструктура района": 0.20,
    "Уровень квартиры": 0.15,
    "Состояние дома": 0.15,
    "Иные факторы": 0.20,
    "Площадь": 0.20  # Увеличенный вес для площади квартиры
}

# Веса атрибутов
weights_of_parameters = {
    "Местоположение": {
        "Район": 0.40,
        "Метро": 0.40,
        "Транспортная доступность": 0.20,
    },
    "Инфраструктура района": {
        "Экология": 0.15,
        "Школы": 0.15,
        "Детские сады": 0.10,
        "Больницы": 0.10,
        "Торговые центры": 0.20,
        "Кафе и рестораны": 0.10,
        "Спорт и отдых": 0.10,
        "Банки": 0.05,
        "Аптеки": 0.05,
    },
    "Уровень квартиры": {
        "Количество комнат": 0.25,
        "Этаж": 0.15,
        "Высота потолков": 0.15,
        "Планировка": 0.25,
        "Состояние": 0.10,
        "Балкон/лоджия": 0.05,
        "Вид из окна": 0.05,
    },
    "Состояние дома": {
        "Тип дома": 0.30,
        "Год постройки": 0.20,
        "Состояние дома": 0.25,
        "Лифт": 0.10,
        "Парковка": 0.10,
        "Консьерж": 0.05,
    },
    "Иные факторы": {
        "Спрос и предложение": 0.40,
        "Экономическая ситуация": 0.30,
        "Политические факторы": 0.20,
        "Сезонность": 0.10
    }
}


def combine_weights(weights_parameters, weights_stations, line, station):
    combined_weights = {}
    station_weight = weights_stations[line].get(station, 0)
    for param, weight in weights_parameters.items():
        combined_weights[param] = weight * (1 + station_weight)
    return combined_weights

def predict_price(base_price, weights_parameters, weights_of_parameters, weights_stations, line, station, apartment_attributes):
    # Получаем вес станции
    station_weight = weights_stations[line].get(station, 0)

    # Комбинируем веса параметров с учетом веса станции
    combined_weights = combine_weights(weights_parameters, weights_stations, line, station)

    # Рассчитываем итоговый вес
    total_weight = 0
    for param, weight in combined_weights.items():
        if param in weights_of_parameters:
            param_weight = 0
            for attr, attr_weight in weights_of_parameters[param].items():
                if attr in apartment_attributes:
                    # Учитываем количественные значения
                    if isinstance(apartment_attributes[attr], (int, float)):
                        param_weight += attr_weight * apartment_attributes[attr] / 5  # Нормируем по 5-балльной шкале
                    else:
                        param_weight += attr_weight * (apartment_attributes[attr] / 5)  # Нормируем по 5-балльной шкале
            total_weight += weight * param_weight
        else:
            total_weight += weight

    # Предсказываем цену
    predicted_price = base_price * (1 + total_weight)

    return predicted_price

# Пример использования
base_price = 15000000  # Базовая цена квартиры
apartment_attributes = {
    "Площадь": 331.5,  # Площадь (в м^2)
    "Район": 3,  # Оценка от 1 до 5
    "Метро": 3,  # Оценка от 1 до 5
    "Транспортная доступность": 4,  # Оценка от 1 до 5
    "Экология": 4,  # Оценка от 1 до 5
    "Школы": 4,  # Оценка от 1 до 5
    "Детские сады": 4,  # Оценка от 1 до 5
    "Больницы": 3,  # Оценка от 1 до 5
    "Торговые центры": 2,  # Оценка от 1 до 5
    "Кафе и рестораны": 2,  # Оценка от 1 до 5
    "Спорт и отдых": 2,  # Оценка от 1 до 5
    "Количество комнат": 1,  # Количество
    "Этаж": 6,  # Этаж квартиры
    "Высота потолков": 2.5,  # Высота (метры)
    "Планировка": 3,  # Оценка от 1 до 5
    "Состояние": 5,  # Оценка от 1 до 5
    "Балкон/лоджия": 1,  # 0 или 1
    "Вид из окна": 3,  # Оценка от 1 до 5
    "Тип дома": 4,  # Оценка от 1 до 5
    "Год постройки": 11,  # Текущая дата - дата продажи (месяцы)
    "Состояние дома": 5,  # Оценка от 1 до 5
    "Лифт": 2,  # Количество
    "Парковка": 1,  # 0 или 1
    "Консьерж": 1,  # 0 или 1
    "Спрос и предложение": 4,  # Оценка от 1 до 5
    "Экономическая ситуация": 3,  # Оценка от 1 до 5
    "Политические факторы": 2,  # Оценка от 1 до 5
    "Сезонность": 3  # Оценка от 1 до 5
}

line = "Розовая"  # Линия метро
station = "Стахановская"  # Станция метро

predicted_price = predict_price(base_price, weight_parameters, weights_of_parameters, weights_stations, line, station, apartment_attributes)
print(f"Предсказанная цена квартиры: {predicted_price:.2f} руб.")

Предсказанная цена квартиры: 28659187.50 руб.
